# SBTi-Finance Tool - Quick Temperature Score Calculation
This notebook provides a simple example of the SBTi-Finance Tool. It shows how to use it to calculate the temperature score for companies, aggregate them to a portfolio level to get the portfolio temperature score. It also shows you how to calculate the portfolio coverage. 

Please see the [methodology](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf), [guidance](https://sciencebasedtargets.org/wp-content/uploads/2020/10/Financial-Sector-Science-Based-Targets-Guidance-Pilot-Version.pdf) and the [technical documentation](https://sciencebasedtargets.github.io/SBTi-finance-tool/) for more details. 

See 1_analysis_example (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/1_analysis_example.ipynb) or [Github](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/master/examples/1_analysis_example.ipynb)) for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular. 

### Install the SBTi Python module
This is only required if you have not already installed the module.

In [ ]:
%pip install sbti-finance-tool

In [ ]:
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
from datetime import datetime
import pandas as pd

## Download the dummy data provider
We have prepared dummy data for you to be able to run the tool as it is to familiarise yourself with how it works. To use your own data; please check out to the [Data Requirements section](https://sciencebasedtargets.github.io/SBTi/DataRequirements.html) of the technical documentation for more details on data requirements and formatting. 

*The dummy data may include some company names, but the data associated with those company names is completely random and any similarities with real world data is purely coincidental. 


In [ ]:
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/example_portfolio.csv", "data/example_portfolio.csv")
if not os.path.isfile("utils.py"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/utils.py", "utils.py")

try:  # Import statement when run in Google Colab
    from utils import anonymize
except:  # Import statement when run locally
    from examples.utils import anonymize

##### Logging
The SBTi module uses the Python standard library logging utilities to send log messages. The log level can be changed according to the user's needs.

In [ ]:
import logging
root_logger = logging.getLogger()
root_logger.setLevel("INFO")

#### Select reporting date for temperature score calculation
Enter the **reporting date** for time-frame classification. Per the CDP/WWF V1 Temperature Rating Methodology
(Section 2.1.4), time frames are "forward looking" relative to the reporting year:
short-term (≤4 years), mid-term (5–15 years), long-term (15–30 years).

This date also determines whether a target is considered "current" (end year >= reporting year).

In [ ]:
year = 2025  # enter the reporting year
month = 12   # enter the month
day = 31     # enter the day

reporting_date = datetime(year, month, day)
print(f"Reporting date: {reporting_date.strftime('%Y-%m-%d')}")

## Create a data provider
Data providers let you connect to the data source of your choice. In this case we are connecting to Excel as a data provider. For all available dataproviders check the implementation [here](https://github.com/ScienceBasedTargets/SBTi-finance-tool/tree/master/SBTi/data)

In [ ]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")

## Load your portfolio
In our case the portfolio is stored as a CSV file. The portfolio should at least have an "id" (the identifier of the company) and a "proportion" (the weight of the company in your portfolio e.g. the value of the shares you hold) column.

Please see the technical documentation on [Data Legends](https://sciencebasedtargets.github.io/SBTi/Legends.html#) for details on data requirements. 

In [ ]:
portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1")
# Rename columns if needed to match expected format
if 'isin' in portfolio.columns and 'company_isin' not in portfolio.columns:
    portfolio = portfolio.rename(columns={'isin': 'company_isin'})
if 'lei' in portfolio.columns and 'company_lei' not in portfolio.columns:
    portfolio = portfolio.rename(columns={'lei': 'company_lei'})

In [ ]:
portfolio.head(5)

To load the data from the data provider, we have to pass a list of IPortfolioCompany instances. The module has a strict [data model](https://sciencebasedtargets.github.io/SBTi-finance-tool/autoapi/SBTi/interfaces/index.html) to convert Pandas Dataframe to the right object types we supplied a utility function.


In [ ]:
companies = SBTi.utils.dataframe_to_portfolio(portfolio)

## Calculate the temperature scores
In the amended portfolio you'll find your original portfolio, amended with the emissions and the temperature score.

In [ ]:
temperature_score = TemperatureScore(                  # all available options:
    time_frames=list(SBTi.interfaces.ETimeFrames),     # ETimeFrames: SHORT MID and LONG
    scopes=[EScope.S1S2, EScope.S3, EScope.S1S2S3],    # EScopes: S3, S1S2 and S1S2S3
    aggregation_method=PortfolioAggregationMethod.WATS # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
)
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies, reporting_date=reporting_date)

For every company the tool assigns a score for all the requested timeframe and scope combinations. In this example we used the full set resulting in 9 scores per company as displayed below:

In [ ]:
amended_portfolio[['company_name', 'time_frame', 'scope', 'temperature_score']].head(9)

## Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done using different aggregation methods. Here we'll use the "Weighted Average Temperature Score" (WATS) by initializing the TemperatureScore Object with PortfolioAggregationMethod.WATS. For more details, please refer to notebook 4 (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/4_portfolio_aggregations.ipynb) or [GitHub](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/master/examples/4_portfolio_aggregations.ipynb)) and the [methodology document](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf) sections 3.2. The temperature scores are calculated per time-frame/scope combination.


In [ ]:
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)

In [ ]:
# Here we cast a ScoreAggregation object to a Pandas Dataframe for viewing the temp scores in a human readable way
pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))

## Portfolio coverage

The portfolio coverage provides insights in the proportion of the portfolio that has set SBTi-approved GHG emissions reduction targets. Only companies with SBTi-status "Approved" are included in the portfolio coverage.

To calculate the portfolio coverage we use the same aggregation methods we use for the Portfolio Temperature Score. In this example we use the "Weighted Average Temperature Score" (WATS). For more details on aggregation methods and the portfolio coverage method, please refer to the [methodology document](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf) sections 3.2 and also turn to notebook 4 (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/4_portfolio_aggregations.ipynb) or [GitHub](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/master/examples/4_portfolio_aggregations.ipynb)) for more aggregation examples.

In [ ]:
portfolio_coverage_tvp = PortfolioCoverageTVP()
coverage = portfolio_coverage_tvp.get_portfolio_coverage(amended_portfolio.copy(), PortfolioAggregationMethod.WATS)
print("Portfolio coverage is: {c:.2f}%".format(c=coverage))

## Export results

You can save the scored portfolio data to an Excel file for further analysis or to share with the SBTi Target Validation Team. Before saving, choose how you want to handle company-identifying information (company name, ISIN, LEI).

### USER INPUT: Choose an anonymization method

Set **ANONYMIZE_MODE** in the next cell to one of three options:

- **"pseudonymize"** -- Replaces real company identifiers with sequential placeholder labels. For example, company names become "Company1", "Company2", etc.; company IDs and ISINs become "C1", "C2", etc.; and LEIs become "L1", "L2", etc. The data relationships are preserved, so the recipient can still reproduce your temperature scores without knowing the real company identities. This is the recommended option when sharing data externally.

- **"remove"** -- Drops the company name, ISIN, and LEI columns entirely from the output. Only the company_id column is kept. Use this if you do not need the recipient to trace scores back to individual companies.

- **"none"** -- Keeps all company-identifying information as-is. Use this if you are saving the file for your own internal records.

In [ ]:
# ---- USER INPUT ----
# Choose one of: "pseudonymize", "remove", or "none"
ANONYMIZE_MODE = "none"
# --------------------

import copy as copy

if ANONYMIZE_MODE == "pseudonymize":
    # Replace identifiers with sequential labels (Company1/C1/L1, Company2/C2/L2, etc.)
    # This preserves data relationships so scores can be reproduced by the recipient.
    portfolio_out = portfolio.copy()
    provider_out = copy.deepcopy(provider)
    portfolio_out, provider_out = anonymize(portfolio_out, provider_out)
    # Map pseudonymized names onto the scored portfolio
    amended_out = amended_portfolio.copy()
    name_map = dict(zip(portfolio['company_name'], portfolio_out['company_name']))
    amended_out['company_name'] = amended_out['company_name'].map(name_map).fillna(amended_out['company_name'])
    if 'company_id' in amended_out.columns and 'company_id' in portfolio_out.columns:
        id_map_dict = dict(zip(portfolio['company_id'], portfolio_out['company_id']))
        amended_out['company_id'] = amended_out['company_id'].map(id_map_dict).fillna(amended_out['company_id'])
    print("Pseudonymized: company identifiers replaced with placeholder labels (Company1, C1, L1, etc.)")

elif ANONYMIZE_MODE == "remove":
    # Strip identifying columns entirely, keeping only company_id
    amended_out = amended_portfolio.copy()
    pii_columns = ['company_name', 'company_isin', 'company_lei']
    columns_to_drop = [col for col in pii_columns if col in amended_out.columns]
    amended_out.drop(columns=columns_to_drop, inplace=True)
    print("Removed: company name, ISIN, and LEI columns dropped from output.")

elif ANONYMIZE_MODE == "none":
    amended_out = amended_portfolio.copy()
    print("No anonymization applied. Output will contain real company identifiers.")

else:
    raise ValueError("ANONYMIZE_MODE must be 'pseudonymize', 'remove', or 'none'.")

In [ ]:
if ANONYMIZE_MODE == "pseudonymize":
    output_filename = 'data/quick_temp_scores_pseudonymized.xlsx'
elif ANONYMIZE_MODE == "remove":
    output_filename = 'data/quick_temp_scores_anonymized.xlsx'
else:
    output_filename = 'data/quick_temp_scores.xlsx'

index_cols = [col for col in ['company_name', 'company_id'] if col in amended_out.columns]
if index_cols:
    amended_out.set_index(index_cols).to_excel(output_filename)
else:
    amended_out.to_excel(output_filename, index=False)

print(f"Saved: {output_filename}")

**Google Colab users:** After running the cell above, click the folder icon in the left pane, open the `data/` folder, hover over the file, click the three dots, and select "Download" to save it to your computer.

**Local users:** The output file will appear in the `data/` folder inside the same directory as this notebook.